In [4]:
%pip uninstall -y trackarray_tensorstore
%pip install -U git+https://github.com/yfukai/tensorstore_trackarr@main

Found existing installation: trackarray-tensorstore 0.0.0
Uninstalling trackarray-tensorstore-0.0.0:
  Successfully uninstalled trackarray-tensorstore-0.0.0
Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/yfukai/tensorstore_trackarr (to revision main) to /private/var/folders/81/kh0zpmbd00g39vwz69lkk4mc0000gn/T/pip-req-build-11_bapv2
  Running command git clone --filter=blob:none --quiet https://github.com/yfukai/tensorstore_trackarr /private/var/folders/81/kh0zpmbd00g39vwz69lkk4mc0000gn/T/pip-req-build-11_bapv2
  Resolved https://github.com/yfukai/tensorstore_trackarr to commit 2833fb29588868bbd830b837dd0a65449fa1e76d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for trackarray-tensorstore: filename=trackarray_tensorstore-0.0.0-py3-none-any.whl size=11897 sha256=b42e472975b86aa04aa179987c0b287d58d640e0c30c424d4bf4e595d14e97d3
  Stored i

In [2]:
!cp "/Volumes/work/fukai/240119-cell-picker/aligned_image5_image_regionprops.parquet" "data/large_sample/"
#!cp "/Volumes/work/fukai//240119-cell-picker/aligned_image5_labels_properties.json" "/Volumes/Extreme SSD/"

In [16]:
image.shape

(1965, 5, 19991, 15437)

In [17]:
image[:20].to_zarr("data/large_sample/aligned_image5_image_20frames.zarr", overwrite=True)
labels[:20].to_zarr("data/large_sample/aligned_image5_labels_20frames.zarr", overwrite=True)
#zarr.save("data/large_sample/aligned_image5_image_20frames.zarr", , overwrite=True)
#zarr.save("data/large_sample/aligned_image5_labels_20frames.zarr", labels[:20], overwrite=True)

KeyboardInterrupt: 

In [3]:
# transform df
from pathlib import Path
import pandas as pd

#bboxes_df_path = Path(r"/Volumes/Extreme SSD/aligned_image5_bboxes_df.csv")
#df = pd.read_csv(bboxes_df_path).reset_index()
#df.to_parquet(bboxes_df_path.with_suffix(".parquet"))

In [ ]:
import pandas as pd
import tensorstore as ts
import json
from copy import deepcopy
import numpy as np
import tensorstore as ts
import trackarray_tensorstore as tats
from pathlib import Path

basedir = "data/large_sample/"

spec = {
    "driver": "zarr3",
    "kvstore": {
            "driver": "file",
            "path": basedir+"aligned_image5_labels.zarr"
    },
    'context': {
            'cache_pool': {
                'total_bytes_limit': 100_000_000
            }
        },
        'recheck_cached_data':False,
    }
labels = ts.open(spec).result()[:,0,:,:]
image_spec = deepcopy(spec)
image_spec['kvstore']['path'] = basedir + "aligned_image5_image.zarr"
image = ts.open(image_spec).result()[:,0,:,:]

print(labels.shape)
bboxes_df_path = Path(basedir+"aligned_image5_bboxes_df.parquet")
properties_path = Path(basedir+"aligned_image5_labels_properties.json")
ta = tats.read_files(labels, bboxes_df_path, properties_path)

(1965, 19991, 15437)


In [6]:
with open(basedir+"aligned_image5_labels_properties.json", "r") as file:
    data = json.load(file)

In [7]:
pickup_sample_df = pd.read_csv("pickup_sample_positions.csv", comment="#")
pickup_sample_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'pickup_sample_positions.csv'

In [ ]:
from napari import Viewer
from napari_travali2._stateful_widget import StateMachineWidget
from napari_travali2._logging import logger
import numpy as np

logger.setLevel("DEBUG")
logger.info("Starting napari-travali2")

viewer = Viewer()
widget = StateMachineWidget(viewer, ta, image, 
                            verified_track_ids=ta.attrs.get("verified_track_ids", []),
                            candidate_track_ids=ta.attrs.get("candidate_track_ids", []),
                            crop_size=2048)
viewer.window.add_dock_widget(widget, area="right")
viewer.dims.set_current_step(0,0)

/Users/fukai/projects/napari-travali2/.venv/lib/python3.13/site-packages/napari/_vispy/layers/scalar_field.py:198: UserWarning: data shape (19991, 15437) exceeds GL_MAX_TEXTURE_SIZE 16384 in at least one axis and will be downsampled. Rendering is currently in 2D mode.
  warnings.warn(
INFO:napari_travali2._logging:Region clicked
INFO:napari_travali2._logging:clicked at [  691 10064  8075]
INFO:napari_travali2._logging:Region selected: coords [  691 10064  8075]
INFO:napari_travali2._logging:window selected: (slice(None, None, None), slice(np.int64(9040), np.int64(11088), None), slice(np.int64(7051), np.int64(9099), None))
INFO:napari_travali2._logging:Track clicked
INFO:napari_travali2._logging:button released
DEBUG:napari_travali2._logging:world coordinates: (np.float64(1066.0), np.float64(9475.985037717906), np.float64(8069.140867638036))
DEBUG:napari_travali2._logging:data coordinates: [1066.          435.98503772 1018.14086764]


: 

In [7]:
pickup_sample_df.head()

,cell ID,tube ID,eject,observed,mosaic Y [px],mosaic X [px],transformed mosaic Y[px],transformed mosaic X[px],misc
0,52,1,2.0,x,5173,6315,5271.203996,6214.819976,15:34
1,56,2,3.0,x,6297,6004,6395.022546,5903.164918,NaN
2,59,3,3.0,o,5691,7648,5789.980833,7547.517842,NaN
3,62,4,3.0,o,6003,6612,6101.376960,6511.336170,NaN
4,68,5,3.0,x,5076,9483,5176.050444,9382.875980,16:23


In [8]:
pickup_sample_df[["transformed mosaic Y[px]","transformed mosaic X[px]"]].values.shape

(39, 2)

In [9]:
pickup_frame = 749 # from /home/fukai/AxioObserver/image_analysis/240119_cellpicker/5_pickup_association/find_pickup_positions_in_stitched_file.ipynb
points = np.hstack((
        np.ones((len(pickup_sample_df),1))*pickup_frame,
        pickup_sample_df[["transformed mosaic Y[px]","transformed mosaic X[px]"]].values))
points[:10]

array([[  749.        ,  5271.20399643,  6214.81997555],
       [  749.        ,  6395.02254562,  5903.16491762],
       [  749.        ,  5789.98083286,  7547.51784227],
       [  749.        ,  6101.37696038,  6511.33617022],
       [  749.        ,  5176.05044369,  9382.87598048],
       [  749.        ,  6477.61717703,  8639.1172497 ],
       [  749.        ,  6109.44778306, 10064.33207839],
       [  749.        ,  6937.88315641,  5663.84847655],
       [  749.        ,  7102.91650086,  7436.75259429],
       [  749.        ,  8331.69131962,  7050.03635092]])

In [10]:
viewer.add_points(points, name="Picked up cells",
                  border_color='blue',
                  size=75, 
                  features=dict(ID=pickup_sample_df["tube ID"]),
                  text="ID",
            face_color="transparent", 
            border_width=0.15)

<Points layer 'Picked up cells' at 0x246ba824890>

In [11]:
@viewer.bind_key("q", overwrite=True)
def go_to_pickup_cell_find_time(viewer):
    viewer.dims.set_current_step(0, pickup_frame)

In [ ]:
viewer.layers.selection 

Selection({<Points layer 'Picked up cells' at 0x246ba824890>})

INFO:napari_travali2._logging:Region clicked
INFO:napari_travali2._logging:clicked at [  749 13853  6262]
INFO:napari_travali2._logging:Region selected: coords [  749 13853  6262]
INFO:napari_travali2._logging:window selected: (slice(None, None, None), slice(np.int64(12829), np.int64(14877), None), slice(np.int64(5238), np.int64(7286), None))
INFO:napari_travali2._logging:Track clicked
INFO:napari_travali2._logging:button released
DEBUG:napari_travali2._logging:world coordinates: (np.float64(749.0), np.float64(13837.397600826751), np.float64(6268.762183142984))
DEBUG:napari_travali2._logging:data coordinates: [ 749.         1008.39760083 1030.76218314]
INFO:napari_travali2._logging:Region clicked
INFO:napari_travali2._logging:clicked at [ 749 5287 6229]
INFO:napari_travali2._logging:Region selected: coords [ 749 5287 6229]
INFO:napari_travali2._logging:window selected: (slice(None, None, None), slice(np.int64(4263), np.int64(6311), None), slice(np.int64(5205), np.int64(7253), None))
IN